### Description

This file calculates metrics for maps on the full reconstruction and temporal deconstructions.

### Inputs

In [ ]:
# =========================================
# For accessing directories
# =========================================
root_dir = "/local/data/artemis/workspace/jfs2167/recon_eval" # Set this to the path of the project

reference_output_dir = f"{root_dir}/references"
recon_output_dir = f"{root_dir}/models/reconstructions"
other_output_dir = f"{root_dir}/models/performance_metrics"

### Modules

In [ ]:
import numpy as np
import pandas as pd
import xarray as xr
from collections import defaultdict
import pickle

### Predefined values

In [ ]:
# Loading references
path_LET = f"{reference_output_dir}/members_LET_dict.pickle"

with open(path_LET,'rb') as handle:
    mems_dict = pickle.load(handle)

In [ ]:
# =========================================
# List of approaches
# =========================================
approaches = ["rf", "xg", "nn"]

# =========================================
# NN reference for selecting correct run to use
# =========================================
val_df_fname = f"{other_output_dir}/nn/nn_test_performance_df.pickle"
nn_val_df = pd.read_pickle(val_df_fname)

### Full reconstruction and temporal deconstruction metric maps

In [ ]:
data_types = ["raw", "seasonal", "decadal", "resid"]

map_data = defaultdict(float) # Data by ML approach
map_data_ens = defaultdict(float) # Data by ML approach and ensemble model

N_ens = 25
N_tot = 100

for ens, mem_list in mems_dict.items():
    print(ens)
    for member in mem_list:
        print(member)
        for approach in approaches:
            recon_dir = f"{recon_output_dir}/{approach}/{ens}/member_{member}"
            
            if approach == "nn":
                run = nn_val_df.query("model == @ens and member == @member and sel_min_bias_mse == 1").index.values[0][2]
                recon_fname_out = f"{recon_dir}/{approach}_recon_temporal_pC02_2D_mon_{ens}_{member}_{run}_1x1_198201-201701.nc"
            else:
                recon_fname_out = f"{recon_dir}/{approach}_recon_temporal_pC02_2D_mon_{ens}_{member}_1x1_198201-201701.nc"
            
            DS_recon = xr.load_dataset(recon_fname_out)
            
            if approach == "rf":
                ref = {}
                ref["raw"] = DS_recon["pCO2"]
                for i in data_types[1:]:
                    ref[i] = DS_recon[f"pCO2_{i}"]
            
            recon = {}
            recon["raw"] = DS_recon["pCO2_recon"]
            for i in data_types[1:]:
                recon[i] = DS_recon[f"pCO2_recon_{i}"]
                
            for i in data_types:
                xmean = ref[i].mean("time")
                ymean = recon[i].mean("time")
                x_minus_mean = ref[i] - xmean
                y_minus_mean = recon[i] - ymean
                ssx = xr.ufuncs.sqrt((x_minus_mean**2).sum("time"))
                ssy = xr.ufuncs.sqrt((y_minus_mean**2).sum("time"))
                
                corr = ( x_minus_mean * y_minus_mean ).sum("time") / (ssx*ssy)
                std_x = ref[i].std("time")
                std_y = recon[i].std("time")
                bias = (ymean - xmean)
                
                # Average bias
                map_data[(approach,i,"bias_mean")] += bias / N_tot
                map_data_ens[(ens,approach,i,"bias_mean")] += bias / N_ens
                
                # Average bias**2
                map_data[(approach,i,"bias_sq")] += bias**2 / N_tot
                map_data_ens[(ens,approach,i,"bias_sq")] += bias**2 / N_ens
                
                # Max bias
                map_data[(approach,i,"bias_max")] = np.maximum(map_data[(approach,i,"bias_max")], bias)
                map_data_ens[(ens,approach,i,"bias_max")] = np.maximum(map_data_ens[(ens,approach,i,"bias_max")], bias)
                
                # Min bias
                map_data[(approach,i,"bias_min")] = np.minimum(map_data[(approach,i,"bias_min")], bias)
                map_data_ens[(ens,approach,i,"bias_min")] = np.minimum(map_data_ens[(ens,approach,i,"bias_min")], bias)
                
                # Mean absolute error
                map_data[(approach,i,"mae")] += np.abs(bias) / N_tot
                map_data_ens[(ens,approach,i,"mae")] += np.abs(bias) / N_ens
                
                # Mean % bias
                map_data[(approach,i,"bias_%error")] += bias/xmean / N_tot
                map_data_ens[(ens,approach,i,"bias_%error")] += bias/xmean / N_ens
                
                # Mean absolute % bias
                map_data[(approach,i,"mae_%error")] += np.abs(bias)/xmean / N_tot
                map_data_ens[(ens,approach,i,"mae_%error")] += np.abs(bias)/xmean / N_ens
                
                # Mean correlation
                map_data[(approach,i,"corr_mean")] += corr / N_tot
                map_data_ens[(ens,approach,i,"corr_mean")] += corr / N_ens
                
                # Average corr**2
                map_data[(approach,i,"corr_sq")] += corr**2 / N_tot
                map_data_ens[(ens,approach,i,"corr_sq")] += corr**2 / N_ens

                # Stdev (amplitude) percentage error
                map_data[(approach,i,"amp_%error")] += (std_y-std_x)/std_x / N_tot
                map_data_ens[(ens,approach,i,"amp_%error")] += (std_y-std_x)/std_x / N_ens

                # Stdev (amplitude) absolute percentage error           
                map_data[(approach,i,"stdev_%error")] += np.abs(std_y-std_x)/std_x / N_tot
                map_data_ens[(ens,approach,i,"stdev_%error")] += np.abs(std_y-std_x)/std_x / N_ens

                
for approach in approaches:
    for i in data_types:
        map_data[(approach,i,"bias_std")] = np.sqrt(map_data[(approach,i,"bias_sq")] - map_data[(approach,i,"bias_mean")]**2)
        map_data[(approach,i,"corr_std")] = np.sqrt(map_data[(approach,i,"corr_sq")] - map_data[(approach,i,"corr_mean")]**2)

        for ens in mems_dict.keys():
            map_data_ens[(ens,approach,i,"bias_std")] = np.sqrt(map_data_ens[(ens,approach,i,"bias_sq")] - map_data_ens[(ens,approach,i,"bias_mean")]**2)
            map_data_ens[(ens,approach,i,"corr_std")] = np.sqrt(map_data_ens[(ens,approach,i,"corr_sq")] - map_data_ens[(ens,approach,i,"corr_mean")]**2)

In [ ]:
map_data_fname = f"{other_output_dir}/map_data_approach.pickle"
map_data_ens_fname = f"{other_output_dir}/map_data_ens.pickle"

with open(map_data_fname,"wb") as handle:
    pickle.dump(map_data, handle)

with open(map_data_ens_fname,"wb") as handle:
    pickle.dump(map_data_ens, handle)